Imports & Data Loading

In [9]:
#Imports
import sys
import os
import pandas as pd

utils_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'utils'))

if utils_path not in sys.path:
    sys.path.insert(0, utils_path)

from create_training_dataset_utils import (load_medcat_export,
    get_validated_entities, get_validated_dates, get_validated_relations, get_validated_relative_dates,
    doc_to_entities_json, doc_to_dates_json, doc_to_relations_json, doc_to_relations_value_json, doc_to_relative_dates_json,
    id2value_from_items, make_row)

In [2]:
#Define cuis for absolute and relative dates (these should align with the cuis that were used to add these terms in MedCAT Trainer)
DATE_CUI = "410671006"
RELATIVE_DATE_CUI = "410671007"

In [5]:
#Load data
export_path = "../data/MedCAT_Export.json"
export = load_medcat_export(export_path)

Explore Data

In [7]:
#Look at data
#export

In [8]:
#Flatten documents
docs = []
for proj in export.get("projects", []):
    for doc in proj.get("documents", []):
        docs.append(doc)

len(docs)

5

In [10]:
#Pick a document to test
doc = docs[0]
#doc

In [13]:
#Get validated clincial entities, absolute dates, relative dates and relations
ents = get_validated_entities(doc, date_cui=DATE_CUI)
dates = get_validated_dates(doc, date_cui=DATE_CUI)
relative_dates = get_validated_relative_dates(doc, relative_date_cui=RELATIVE_DATE_CUI)
relations = get_validated_relations(doc, date_cui=DATE_CUI, relative_date_cui=RELATIVE_DATE_CUI)

print("Clincial entities:", len(ents))
print("Absolute dates:", len(dates))
print("Relative dates:", len(relative_dates))
print("Relations:", len(relations))

Clincial entities: 64
Absolute dates: 6
Relative dates: 0
Relations: 4


In [14]:
#Look at first 5 of each
ents[:5], dates[:5], relative_dates[:5], relations[:5]

([{'id': 308244,
   'value': 'history of meningitis',
   'cui': '161478002',
   'start': 756,
   'end': 777},
  {'id': 308245,
   'value': 'rheumatoid_arthritis',
   'cui': '69896004',
   'start': 491,
   'end': 511},
  {'id': 308248,
   'value': 'current medication',
   'cui': '513881000000106',
   'start': 145,
   'end': 163},
  {'id': 308249,
   'value': 'normal vital signs',
   'cui': '72970002',
   'start': 289,
   'end': 307},
  {'id': 308252,
   'value': 'pituitary_adenoma',
   'cui': '254956000',
   'start': 410,
   'end': 427}],
 [{'id': 308320, 'value': '30nd Jun 2024', 'start': 12, 'end': 25},
  {'id': 308321, 'value': '12nd Sep 2024', 'start': 363, 'end': 376},
  {'id': 308322, 'value': "16 Sep'24", 'start': 443, 'end': 452},
  {'id': 308323, 'value': '23rd Oct 2024', 'start': 588, 'end': 601},
  {'id': 308324, 'value': '16st Nov 2024', 'start': 1205, 'end': 1218}],
 [],
 [{'date_id': 308321, 'entity_id': 308252},
  {'date_id': 308322, 'entity_id': 308245},
  {'date_id': 30

In [20]:
# Make relations human-readable (map IDs to values)
id2value = id2value_from_items(ents, dates, relative_dates)

readable_relations = [
    {
        "date_value": id2value.get(L["date_id"]),
        "entity_value": id2value.get(L["entity_id"]),
    }
    for L in relations
]
readable_relations

[{'date_value': '18/11/24', 'entity_value': 'nausea/vomiting'},
 {'date_value': '18/11/24',
  'entity_value': 'history of neoplasm of pituitary gland'},
 {'date_value': '2024-12-14', 'entity_value': 'improved'},
 {'date_value': '12/02/25', 'entity_value': 'diabetes_mellitus'}]

In [16]:
#Run across all documents and summarize counts
summary = []
for d in docs:
    summary.append({
        "doc_id": d.get("id"),
        "n_entities": len(get_validated_entities(d, DATE_CUI)),
        "n_dates": len(get_validated_dates(d, DATE_CUI)),
        "n_rel_dates": len(get_validated_relative_dates(d, RELATIVE_DATE_CUI)),
        "n_relations": len(get_validated_relations(d, DATE_CUI, RELATIVE_DATE_CUI)),
    })

pd.DataFrame(summary).sort_values("doc_id").reset_index(drop=True)

,doc_id,n_entities,n_dates,n_rel_dates,n_relations
0,26461,64,6,0,4
1,26462,21,7,0,11
2,26463,15,7,0,12
3,26464,11,3,0,6
4,26465,24,3,0,4


Create Dataset

In [17]:
#Extract validated entities, dates, relative dates and relations from all docs in dataset and append to rows
rows = []
for doc in docs:
    # Extract entities, dates, relative dates and relations
    ents = get_validated_entities(doc, DATE_CUI)
    dates = get_validated_dates(doc, DATE_CUI)
    relative_dates = get_validated_relative_dates(doc, RELATIVE_DATE_CUI)
    relations = get_validated_relations(doc, DATE_CUI, RELATIVE_DATE_CUI)

    # Mapping from validated items
    id2value = id2value_from_items(ents, dates, relative_dates)

    # Serialize each field
    entities_json = doc_to_entities_json(ents)
    dates_json = doc_to_dates_json(dates)
    relative_dates_json = doc_to_relative_dates_json(relative_dates)
    relations_json = doc_to_relations_value_json(relations, id2value)

    # Assemble row
    rows.append(make_row(
        doc_id=doc.get("id"),
        note_text=doc.get("text", ""),
        entities_json=entities_json,
        dates_json=dates_json,
        relative_dates_json=relative_dates_json,
        relations_json=relations_json
    ))

In [18]:
#Create dataframe
df = pd.DataFrame(rows, columns=["doc_id", "note_text", "entities_json", "dates_json", "relative_dates_json", "relations_json"])
df

,doc_id,note_text,entities_json,dates_json,relative_dates_json,relations_json
0,26461,Ultrasound (30nd Jun 2024): no significant fin...,"[{""id"": 308244, ""value"": ""history of meningiti...","[{""id"": 308320, ""value"": ""30nd Jun 2024"", ""sta...",[],"[{""date"": ""12nd Sep 2024"", ""entity"": ""pituitar..."
1,26462,Labs (27th Sep 2024): anemia. resolving Skin:...,"[{""id"": 308371, ""value"": ""lesions"", ""cui"": ""52...","[{""id"": 308581, ""value"": ""22/11/24"", ""start"": ...",[],"[{""date"": ""27th Sep 2024"", ""entity"": ""anemia"",..."
2,26463,URGENT REVIEW (2024-10-04): cough. suspect ost...,"[{""id"": 308886, ""value"": ""frequent urination"",...","[{""id"": 308940, ""value"": ""2024-10-04"", ""start""...",[],"[{""date"": ""2024-10-04"", ""entity"": ""cough"", ""da..."
3,26464,URGENT REVIEW (13rd Feb 2025) MRI of the brain...,"[{""id"": 308951, ""value"": ""multiple_sclerosis"",...","[{""id"": 308996, ""value"": ""05-03-2025"", ""start""...",[],"[{""date"": ""13rd Feb 2025"", ""entity"": ""visual"",..."
4,26465,New pt((18/11/24)): pt presents with nausea/vo...,"[{""id"": 308998, ""value"": ""history of neoplasm ...","[{""id"": 309070, ""value"": ""18/11/24"", ""start"": ...",[],"[{""date"": ""18/11/24"", ""entity"": ""nausea/vomiti..."


In [19]:
#Save csv
out_path = "../data/training_dataset.csv"
df.to_csv(out_path, index=False)